In [181]:
# 1. Drop pairs that do not exist in test set
# 2. Train another model for them
# 3. Train another models for low-rowed train set

### 1

In [291]:
train_split_date = '2019-10-01'

df_merged_train = df_merged[df_merged['period_start_dt'] <= train_split_date]
df_merged_test = df_merged[df_merged['period_start_dt'] > train_split_date]


print(f'Train contains {round(df_merged_train.shape[0] / df_merged.shape[0], 2) * 100}% of data - {df_merged_train.shape[0]} rows')
print(f'Test contains {round(df_merged_test.shape[0] / df_merged.shape[0], 2) * 100}% of data - {df_merged_test.shape[0]} rows')

Train contains 80.0% of data - 1053453 rows
Test contains 20.0% of data - 255403 rows


In [292]:
# df_merged_train = df_merged_train.drop('key', axis=1)

In [293]:
test_pairs = df_merged_test.groupby(['product_rk', 'store_location_rk'])['demand'].count().\
    reset_index().rename(columns={'demand': 'key'})

test_pairs['key'] = range(test_pairs.shape[0])

In [294]:
test_pairs

,product_rk,store_location_rk,key
0,11478,1034,0
1,11478,1281,1
2,11506,764,2
3,11506,1034,3
4,11506,1281,4
...,...,...,...
58945,147201,1143,58945
58946,147201,1173,58946
58947,147201,1281,58947
58948,147201,1380,58948


In [295]:
# df_merged_test = df_merged_test.drop('key', axis=1)

In [296]:
df_merged_test = df_merged_test.merge(test_pairs, how='left', on=['product_rk', 'store_location_rk'])


df_merged_train = df_merged_train.merge(test_pairs, how='left', on=['product_rk', 'store_location_rk'])
df_merged_train = df_merged_train.dropna(subset=['key'])


In [297]:
df_merged_train['key'].unique()[5998]

20854.0

In [298]:
df_merged_train[df_merged_train['key'] == 20854.0]

,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,...,demand_lag52_w224D_keyproduct_rk_store_location_rk_preagmean_agmean_NO_FILTER_dynamic_rolling,demand_lag52_w224D_keyproduct_rk_store_location_rk_preagmean_agmedian_NO_FILTER_dynamic_rolling,demand_lag52_w728D_keyproduct_rk_store_location_rk_preagmean_agmean_NO_FILTER_dynamic_rolling,demand_lag52_w728D_keyproduct_rk_store_location_rk_preagmean_agmedian_NO_FILTER_dynamic_rolling,demand_lag52_w1456D_keyproduct_rk_store_location_rk_preagmean_agmean_NO_FILTER_dynamic_rolling,demand_lag52_w1456D_keyproduct_rk_store_location_rk_preagmean_agmedian_NO_FILTER_dynamic_rolling,city,region,label,key
14037,80394,1034,2018-10-08,0.044444,0.0,0.0,4149.0,2696.95,0.0,1.0,...,0.531343,0.41791,0.496546,0.41791,0.496546,0.41791,203,10,0,20854.0


In [299]:
test = df_merged_train[df_merged_train['key'] == 20854.0]

In [300]:
test_new = test.copy()
test_new['demand'] += np.random.normal(scale=1e-4, size=test.shape[0])
pd.concat([test, test_new], axis=0)

,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,...,demand_lag52_w224D_keyproduct_rk_store_location_rk_preagmean_agmean_NO_FILTER_dynamic_rolling,demand_lag52_w224D_keyproduct_rk_store_location_rk_preagmean_agmedian_NO_FILTER_dynamic_rolling,demand_lag52_w728D_keyproduct_rk_store_location_rk_preagmean_agmean_NO_FILTER_dynamic_rolling,demand_lag52_w728D_keyproduct_rk_store_location_rk_preagmean_agmedian_NO_FILTER_dynamic_rolling,demand_lag52_w1456D_keyproduct_rk_store_location_rk_preagmean_agmean_NO_FILTER_dynamic_rolling,demand_lag52_w1456D_keyproduct_rk_store_location_rk_preagmean_agmedian_NO_FILTER_dynamic_rolling,city,region,label,key
14037,80394,1034,2018-10-08,0.044444,0.0,0.0,4149.0,2696.95,0.0,1.0,...,0.531343,0.41791,0.496546,0.41791,0.496546,0.41791,203,10,0,20854.0
14037,80394,1034,2018-10-08,0.044286,0.0,0.0,4149.0,2696.95,0.0,1.0,...,0.531343,0.41791,0.496546,0.41791,0.496546,0.41791,203,10,0,20854.0


In [304]:
# test = df_merged_train[df_merged_train['key'] == 21934.0]

# if (test['demand'] == test['demand'].iloc[0]).all():
#     test.loc['demand'] += np.random.normal(scale=1e-4, size=test.shape[0])

In [312]:
n_samples = df_merged_train.groupby('key')['demand'].count().reset_index().rename(columns={'demand': 'n_train'})
df_merged_train = df_merged_train.merge(n_samples, on='key', how='left')

In [322]:
key2keep = df_merged_train[df_merged_train['n_train'] > 40]['key'].unique()

In [323]:
trainSet = df_merged_train[df_merged_train['key'].isin(key2keep)]
testSet = df_merged_test[df_merged_test['key'].isin(key2keep)]

In [324]:
mse_sum = 0
shape_sum = 0
main_cols = ['product_rk', 'store_location_rk', 'PROMO1_FLAG', 'PROMO2_FLAG', 'city']
lag_cols = trainSet.columns[trainSet.columns.str.contains('demand_')]
train_cols = main_cols + list(lag_cols)

for key_pair in tqdm(trainSet['key'].unique(), total=len(trainSet['key'].unique())):
    train = trainSet[trainSet['key'] == key_pair]
    
#     сделал выше
#     if train.shape[0] < 30:
#         continue
        
    if train.shape[0] == 1:
        continue
#         train_tmp = train.copy()
#         train_tmp['demand'] += np.random.normal(scale=1e-4, size=1)
#         train = pd.concat([train, train_tmp], axis=0)
        
    if (train['demand'] == train['demand'].iloc[0]).all():
        train['demand'] += np.random.normal(scale=1e-4, size=train.shape[0])
        
    X_train, y_train = train[train_cols].values, train['demand'].values
    X_test, y_test = testSet[testSet['key'] == key_pair][train_cols].values, testSet[testSet['key'] == key_pair]['demand'].values
    
#     best_params = get_best_params(X_train, y_train)

#     cat = CatBoostRegressor(iterations=100, **best_params)
    cat = CatBoostRegressor(iterations=100, verbose=0)
    cat.fit(X_train, y_train)
    
    preds = cat.predict(X_test)
    
    mse_sum += mean_squared_error(preds, y_test) * X_test.shape[0]
    shape_sum += X_test.shape[0]
    

  0%|          | 0/1188 [00:00<?, ?it/s]

In [325]:
mse_sum / shape_sum

50.32074226733585

In [233]:
# store = 926
# product = 45312


# df[(df['product_rk'] == product) & (df['store_location_rk'] == store)].sort_values('period_start_dt').\
# set_index('period_start_dt')[['demand']].plot(figsize=(10, 8), color='blue', label=f'Demand_Store{store}_product{product}')
# plt.xlabel('Date', weight='bold', size=12)
# plt.ylabel('Demand', weight='bold', size=12)
# plt.legend()
